In [1]:
import sys
sys.path.append("/home/user/backup/")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import math
import tensorflow as tf
import utils
from timegan import *

2023-02-23 07:43:22.076487: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def load_all_data(dir, filecnt=54):
    data = [None]*10
    t = np.zeros((10, filecnt), int)
    for p in range(10):
        d = [None]*filecnt
        for i in range(filecnt):
            d[i] = np.load(f"{dir}/{p}/{i+1}.npy")
            t[p,i] = d[i].shape[0]
        data[p] = d
    return data, t

def load_with_config(dir, config_path, load_nonvalid=False):
    config_arr = np.array(pd.read_csv(f"{config_path}", header=None, skiprows=1))
    data = [[] for i in range(10)]          #create empty 2d list : (10, unknown)
    for config in config_arr:
        #valid check
        if (not config[2]) and (not load_nonvalid):
            continue
        data[int(config[0])] += [np.load(f"{dir}/{int(config[0])}/{int(config[1])+1}.npy")[int(config[3]):int(config[4]),:]]
    
    return data, config_arr

def match_length(d, t:int):
    """
    return
        x : (N, times(t), 2) shape numpy array,
        y : (N)
    """
    N = sum([len(d[i]) for i in range(len(d))])
    x = np.zeros((N, t, 2), np.float64)
    y = np.zeros((N), np.float64)
    target_timepoints = np.linspace(0, 1, t)
    start_at = 0
    for r in range(len(d)):
        for c in range(len(d[r])):
            origin_timepoints = np.linspace(0, 1, d[r][c].shape[0])
            x[start_at + c, :, 0] = np.interp(target_timepoints, origin_timepoints, d[r][c][:,0])
            x[start_at + c, :, 1] = np.interp(target_timepoints, origin_timepoints, d[r][c][:,1])
        y[start_at:start_at + len(d[r])] = r
        start_at += len(d[r])
    return x, y 

def apply_normalize(d):
    for r in range(len(d)):
        for c in range(len(d[r])):
            channels = d[r][c].shape[1]
            min_vals = np.min(d[r][c][:, :], axis=0)
            max_vals = np.max(d[r][c][:, :], axis=0)
            min_max_diff = np.array([max_vals[j] - min_vals[j] for j in range(channels)])
            factor = min_max_diff / np.max(min_max_diff)
            for ch in range(channels):
                d[r][c][:, ch] = ((d[r][c][:, ch] - min_vals[ch])/min_max_diff[ch])*factor[ch]
    
def plot_data(d):
    plt.figure(figsize=(20,4))
    plt.subplot(2,1,1)
    gca = plt.gca()
    gca.plot(d[:,0])
    plt.subplot(2,1,2)
    gca = plt.gca()
    gca.plot(d[:,1])
    plt.tight_layout()
    plt.show()

def plot_data2(d, save=None, cmap_name="gist_rainbow", xlim1=None, ylim1=None, xlim2=None, ylim2=None, xlim3=None, ylim3=None):
    fig, axes = plt.subplot_mosaic("abbbb;acccc", figsize=(20,4))
    utils.draw_gradation(d[:,0], d[:,1], axes["a"],cmap_name=cmap_name, xlim=xlim1, ylim=ylim1)
    utils.draw_gradation(np.arange(d.shape[0]), d[:,0], axes["b"],cmap_name=cmap_name, xlim=xlim2, ylim=ylim2)
    utils.draw_gradation(np.arange(d.shape[0]), d[:,1], axes["c"],cmap_name=cmap_name, xlim=xlim3, ylim=ylim3)

    plt.tight_layout()
    if save is None:
        plt.show()
    else:
        plt.savefig(save)
        plt.close()

def plot_data3(origin, syn, save=None, cmap_name="gist_rainbow", hightitle="Original", lowtitle="Synthetic"):
    fig, axes = plt.subplot_mosaic("abbbb;acccc;deeee;dffff", figsize=(20,8))
    utils.draw_gradation(origin[:,0], origin[:,1], axes["a"],cmap_name=cmap_name, xlim=[0,1], ylim=[0,1])
    utils.draw_gradation(np.arange(origin.shape[0]), origin[:,0], axes["b"],cmap_name=cmap_name, ylim=[0,1])
    utils.draw_gradation(np.arange(origin.shape[0]), origin[:,1], axes["c"],cmap_name=cmap_name, ylim=[0,1])
    utils.draw_gradation(syn[:,0], syn[:,1], axes["d"], cmap_name=cmap_name, xlim=[0,1], ylim=[0,1])
    utils.draw_gradation(np.arange(syn.shape[0]), syn[:,0], axes["e"],cmap_name=cmap_name, ylim=[0,1])
    utils.draw_gradation(np.arange(syn.shape[0]), syn[:,1], axes["f"],cmap_name=cmap_name, ylim=[0,1])
    axes["a"].set_title(hightitle)
    axes["d"].set_title(lowtitle)

    plt.tight_layout()
    if save is None:
        plt.show()
    else:
        plt.savefig(save)
        plt.close()

def plot_data4(datas, save=None, titles=None, cmap_name="winter_r"):
    plotlen = len(datas)
    mosaic_input = [None] * (plotlen << 1)
    for i in range(plotlen):
        j = i << 1
        mosaic_input[j] = [f"{i}a"] + [f"{i}x"] * 4
        mosaic_input[j+1] = [f"{i}a"] + [f"{i}y"] * 4
    fig, axes = plt.subplot_mosaic(mosaic_input, figsize=(20, 4 * plotlen))
    for i in range(plotlen):
        if titles is not None:
            axes[f"{i}a"].set_title(titles[i])
        utils.draw_gradation(datas[i][:,0], datas[i][:,1], axes[f"{i}a"], cmap_name=cmap_name, xlim=[0,1], ylim=[0,1])
        x_indexes = np.arange(datas[i].shape[0])
        utils.draw_gradation(x_indexes, datas[i][:,0], axes[f"{i}x"], cmap_name=cmap_name, ylim=[0,1])
        utils.draw_gradation(x_indexes, datas[i][:,1], axes[f"{i}y"], cmap_name=cmap_name, ylim=[0,1])

    plt.tight_layout()
    if save is None:
        plt.show()
    else:
        plt.savefig(save)
        plt.close()


In [4]:
data, config = load_with_config("/home/user/workspace/research/eye-writing/self_data/", "/home/user/workspace/research/eye-writing/load_data_config.csv", load_nonvalid=False)
apply_normalize(data)

In [5]:
x, y = match_length(data, 200)
print(x.shape)
print(y.shape)
pattern = 0
x = x[y==pattern]
print(x.shape)


(505, 200, 2)
(505,)
(51, 200, 2)


In [10]:
rnn_units = 24
rnn_layers = 3
batch_size = 32
lr = 0.0005
epoch = 2000
save_name = f"/home/user/backup/synckpt_test_{pattern}/"
import os

z = np.random.uniform(size=(batch_size, x.shape[1], x.shape[2]))

if os.path.isdir(save_name):
    print(f"체크포인트 있음, 불러옴.")
    syn_gen = generator_load(save_name, x.shape[1], x.shape[2], rnn_units, rnn_layers)
else:
    timegan_tuple = timegan_init(x.shape[1], x.shape[2], rnn_units, rnn_layers)
    timegan_tuple = timegan_train(x, timegan_tuple, epochs=epoch, batch_size=batch_size, learning_rate=lr)
    syn_gen = timegan_export_generator(timegan_tuple)
    # generator_save(syn_gen, save_name)


2023-02-22 07:40:31.613251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-22 07:40:31.626874: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-22 07:40:31.627133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-22 07:40:31.628195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

train autoencoder


  0%|          | 0/2000 [00:00<?, ?it/s]2023-02-22 07:40:36.774208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-22 07:40:37.841566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-22 07:40:38.497185: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x6bff4010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-22 07:40:38.497247: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2023-02-22 07:40:38.528720: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-22 07:40:38.772999: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is l

100%|██████████| 2000/2000 [05:03<00:00,  6.60it/s]


joint train


 53%|█████▎    | 1067/2000 [27:19<23:00,  1.48s/it]

In [ ]:
# synth_data = generator_gen(syn_gen, batch_size)
# synth_data.shape
e, g, s, r ,d = timegan_tuple
# synth_data = e(x)
# synth_data = r(synth_data)
# synth_data = g(z)
# synth_data = s(synth_data)

e_l = e(x)
e_l_r = r(e_l)
g_l = g(z)
g_l_r = r(g_l)

In [ ]:
save_dir = f"/home/user/output"

import os
import glob

files = glob.glob(f'{save_dir}/*.png')
for f in files:
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

# for i in range(len(synth_data)):
#     plot_data3(x[i], synth_data[i], f"{save_dir}/{i}.png", "winter_r")

for i in range(batch_size):
    plot_data4(datas=[x[i], e_l[i], e_l_r[i], g_l[i], g_l_r[i]],
     save=f"{save_dir}/{i}.png", 
     titles=["x", "x -> latent code", "reconstructed data", "z -> latent code", "z -> reconstrcution"])